In [1]:
import pandas as pd

import pathlib

import xlrd

In [5]:
ruta_git = pathlib.Path().cwd().parent.parent.parent

obs = pd.read_excel(ruta_git/'data/raw/obs_1006.xlsx')
obs

,Iniciar,Enviar,File,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_29_3,x_40_1,x_40_2,x_40_3,x_40_4,x_40_5,x_40_6,IdEncuesta,Usuario,Fecha
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Iniciar Encuesta Ahora,EnvioEncuesta,DSD,2021-12-31,SDSD,1.0,DDSDS,DSDS,6,1,...,--,--,3,--,5,dsdsds,2,cristianavella,2022-03-08 16:36:06,NaN
2,Iniciar Encuesta Ahora,EnvioEncuesta,TESR,2021-12-31,JONATHAN RAMÍREZ,101010.0,PROCUREMENT,MATEMÁTICAS,11,2,...,1,--,--,4,--,Prueba de cierre,3,lorenamolina,2022-03-08 16:19:31,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Iniciar Encuesta Ahora,EnvioEncuesta,JOHANA DE HOYOS,2022-05-17,IVÁN RÍOS BLANCO,92539680.0,TECNOLOGÍA E INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,9,2,...,NaN,--,--,--,--,--,No se realiza ninguna acción sobre equidad de ...,226,ing.johanadehoyos@gmail.com,2022-05-20 06:25:58
208,Iniciar Encuesta Ahora,EnvioEncuesta,MÓNICA YAZMÍN GIRALDO OSORIO,20/05/2022,LUIS GUILLERMO ZULETA,71773232.0,TECNOLOGÍA E INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,10,3,...,NaN,--,--,--,--,--,"En la pregunta 40, no se da ninguna de las opc...",227,mkgiraldoosorio@gmail.com,2022-05-20 13:10:53
209,Iniciar Encuesta Ahora,EnvioEncuesta,JOHANA DE HOYOS,2022-05-18,GUILLERMO HERNANDEZ HERNANDEZ,92529908.0,TECNOLOGÍA E INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,11,4,...,NaN,--,--,--,--,--,No se hace ningún comentario del tema.,228,ing.johanadehoyos@gmail.com,2022-05-20 19:17:48
210,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
obs = obs.dropna(how='all')
obs

,Iniciar,Enviar,File,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_29_3,x_40_1,x_40_2,x_40_3,x_40_4,x_40_5,x_40_6,IdEncuesta,Usuario,Fecha
1,Iniciar Encuesta Ahora,EnvioEncuesta,DSD,2021-12-31,SDSD,1.0,DDSDS,DSDS,6,1,...,--,--,3,--,5,dsdsds,2,cristianavella,2022-03-08 16:36:06,NaN
2,Iniciar Encuesta Ahora,EnvioEncuesta,TESR,2021-12-31,JONATHAN RAMÍREZ,101010.0,PROCUREMENT,MATEMÁTICAS,11,2,...,1,--,--,4,--,Prueba de cierre,3,lorenamolina,2022-03-08 16:19:31,NaN
16,Iniciar Encuesta Ahora,EnvioEncuesta,MIRYAM VIVIANA RODRÍGUEZ,2022-03-24,JAVIER BUSTAMANTE HERRERA,75102867.0,INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,11,1,...,--,--,--,--,--,"No se habló de equidad de género, ni se foment...",21,vivirodriguezv16@gmail.com,2022-03-24 20:27:31,NaN
30,Iniciar Encuesta Ahora,EnvioEncuesta,CRISTIAN AVELLA,2022-03-23,JUAN CARLOS SANABRIA ROMERO,104977766.0,INGLÉS,LENGUA EXTRANJERA,4,5,...,--,2,--,--,--,NaN,35,jhon.balcarcel@gmail.com,2022-03-23 11:49:42,NaN
34,Iniciar Encuesta Ahora,EnvioEncuesta,MÓNICA YAZMÍN GIRALDO OSORIO,24/03/2022,CAMILO JAVIER BUSTAMANTE HEREDIA,75102867.0,TECNOLOGÍA E INFORMÁTICA,NaN,11,1,...,--,--,--,4,--,NaN,39,mkgiraldoosorio@gmail.com,2022-03-24 18:42:49,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,Iniciar Encuesta Ahora,EnvioEncuesta,CÉSAR AUGUSTO GAVIRIA HERRERA,2022-04-26,JOSÉ RICARDO PÉREZ CANTILLO,77165032.0,TECNOLOGÍA E INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,8,1,...,NaN,--,--,--,--,--,No se hace uso de esta estrategia.,225,gaviriace16@gmail.com,2022-05-18 12:55:15
207,Iniciar Encuesta Ahora,EnvioEncuesta,JOHANA DE HOYOS,2022-05-17,IVÁN RÍOS BLANCO,92539680.0,TECNOLOGÍA E INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,9,2,...,NaN,--,--,--,--,--,No se realiza ninguna acción sobre equidad de ...,226,ing.johanadehoyos@gmail.com,2022-05-20 06:25:58
208,Iniciar Encuesta Ahora,EnvioEncuesta,MÓNICA YAZMÍN GIRALDO OSORIO,20/05/2022,LUIS GUILLERMO ZULETA,71773232.0,TECNOLOGÍA E INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,10,3,...,NaN,--,--,--,--,--,"En la pregunta 40, no se da ninguna de las opc...",227,mkgiraldoosorio@gmail.com,2022-05-20 13:10:53
209,Iniciar Encuesta Ahora,EnvioEncuesta,JOHANA DE HOYOS,2022-05-18,GUILLERMO HERNANDEZ HERNANDEZ,92529908.0,TECNOLOGÍA E INFORMÁTICA,TECNOLOGÍA E INFORMÁTICA,11,4,...,NaN,--,--,--,--,--,No se hace ningún comentario del tema.,228,ing.johanadehoyos@gmail.com,2022-05-20 19:17:48
